In [2]:
from rr_python_sdk import cpp, MeasurementHead

In [3]:
head = MeasurementHead("COM5")

In [5]:
head.home

In [6]:
payload = cpp.JogPayload
payload.distance_mm = 1.0
head.jog_motor(payload)

ValueError: zero_forces can only be called with an argument of type EmptyPayload

In [10]:
locals()['hs']

In [11]:
my_head = head.MeasurementHead("COM5")

SerialException: could not open port 'COM5': PermissionError(13, 'Access is denied.', None, 5)

In [3]:
my_head.ack()

ACK b'\xaa\xaa\x06\x816'


In [5]:
my_head.read()

AttributeError: type object 'MeasurementHead' has no attribute 'ser'

In [6]:
head.cpp.AccelerationData()

SyntaxError: invalid syntax (3813758584.py, line 1)

In [ ]:
head.cpp.MsgType_ToHost.

In [26]:
# --- Force Data Point ---
force = cpp.ForceDataPoint()  # corresponds to force_data_point_t
force.device_status = 2
force.f_normal_mn = 123456
force.f_tangential_mn = -654321
force.millis_since_start = 500

# Serialize to bytes
# We'll use memoryview/bytearray approach
force_bytes = force.serialize()

# Prepend tag for ESP (0x00)
frame = bytes([0x00]) + force_bytes
ser.write(frame)
print("Sent force_data_point_t: 0x" + ' 0x'.join(f'{x:02X}' for x in frame)[0:])

Sent force_data_point_t: 0x00 0x02 0x00 0x00 0x00 0x40 0xE2 0x01 0x00 0x0F 0x04 0xF6 0xFF 0xF4 0x01


In [25]:
# AccelerationData
accel = cpp.AccelerationData()
accel.x = 1.1
accel.y = 2.2
accel.z = 3.3
accel.t = 4.4

accel_bytes = accel.serialize()
ser.write(bytes([0x01]) + accel_bytes)
print("Sent acceleration_data_t:", accel_bytes)

Sent acceleration_data_t: b'\xcd\xcc\x8c?\xcd\xcc\x0c@33S@\xcd\xcc\x8c@'


In [35]:
payload = b"Hello ESP32!\r\n"

print(f"Sending {len(payload)} bytes: {payload}")
ser.write(payload)

# Read back the same number of bytes
rx = ser.read(len(payload))
print(f"Received {len(rx)} bytes: {rx}")

# Compare
if rx == payload:
    print("✅ Echo OK")
else:
    print("❌ Mismatch!")
    print("Sent :", binascii.hexlify(payload))
    print("Recv :", binascii.hexlify(rx))

Sending 14 bytes: b'Hello ESP32!\r\n'
Received 14 bytes: b'Hello ESP32!\r\n'
✅ Echo OK


'00 0x02 0x00 0x00 0x00 0x40 0xE2 0x01 0x00 0x0F 0x04 0xF6 0xFF 0xF4 0x01'

In [37]:
ser.close()

In [ ]:
head = sdk.MeasurmentHead(port = "COM11")
head.read_all_input()

# Test all functions
 (In order they are in the microcontroller code)

## Angle setting

In [ ]:
head.set_angle(10)

## Echo input

In [ ]:
head.echo(b"Hello")

## Read and write config data
Note, the default config will be reloaded when the device restarts

In [ ]:
config = head.read_config_data()
config.microsteps_per_step

In [ ]:
config.t_force_scale

In [ ]:
config.microsteps_per_step = 8

In [ ]:
head.update_config_data(config)

In [ ]:
config = head.read_config_data()

In [ ]:
config.microsteps_per_step

## Data streaming

In [ ]:
i = 0
points = []
head.read_all_input()
head.start_streaming_force_data(100)
while (i<200):
    points.append(head.read_force_data_point_from_stream())
    i+=1
head.stop_streaming_force_data()
#plt.plot([p.ms_since_start for p in points], [p.f_t_mn/p.f_n_mn for p in points], label="mu")
plt.plot([p.ms_since_start for p in points], [p.f_n_mn for p in points], label="Normal force (mn)")
plt.plot([p.ms_since_start for p in points], [p.f_t_mn for p in points], label="Tangential force (mn)")
plt.legend()

In [ ]:
head.stop_streaming_force_data()
head.read_all_input()

## Show and reset error data

In [ ]:
f"{head.read_error_status():016b}"

In [ ]:
head.clear_error_status()

## Setting target force

In [ ]:
head.set_target_force(100000)

## Home

In [ ]:
head.home()

## Jog

In [ ]:
head.jog(100)

## Toggle LEDS

In [ ]:
head.toggle_led(True)
time.sleep(0.5)
head.toggle_led(False)
time.sleep(0.5)
head.toggle_led(True)
time.sleep(0.5)
head.toggle_led(False)

## Change state to None

In [ ]:
head.stop_all()

## Send current data

In [ ]:
head.read_single_force_data_point()

## Restart

In [ ]:
head.restart()

In [ ]:
head.read_all_input()

## Start test

In [ ]:
head.maintain_force(100000)

In [ ]:
i = 0
points = []
head.read_all_input()
head.start_streaming_force_data(100)
while (i<200):
    points.append(head.read_force_data_point_from_stream())
    i+=1
head.stop_streaming_force_data()

centre=0
vals = np.sort([p.f_t_mn for p in points])
span = np.mean(vals[-50:])-np.mean(vals[:50])
plt.plot([p.ms_since_start for p in points], [p.f_n_mn for p in points], label="Normal force (mn)")
plt.plot([p.ms_since_start for p in points], [p.f_t_mn for p in points], label="Tangential force (mn)")
plt.plot([0,2000], [centre, centre], 'k:')
plt.plot([0,2000], [centre-span/2, centre-span/2], 'g-')
plt.plot([0,2000], [centre+span/2, centre+span/2], 'g-')
plt.legend()

In [ ]:
head.stop_all()

## Zero all forces

In [ ]:
head.home()

In [ ]:
head.zero_forces()

## Incline and roll

In [ ]:
head.read_all_input()
head.read_orientation_data()